# 目的
debertaでの学習を行う

ref: https://www.kaggle.com/code/fritzcremer/lmsys-deberta-v3-base-baseline

In [1]:
# path setting
EXP_NAME = "e001-baseline"
MODEL_NAME = "microsoft/deberta-v3-xsmall"
COMPETITION_NAME = "eedi-mining-misconceptions-in-mathematics"

DATA_PATH = "retrieved_data"
ENV_PATH = "env_file"
DATASET_NAME = f"{EXP_NAME}-{MODEL_NAME.split('/')[-1]}"
MODEL_OUTPUT_PATH = f"{COMPETITION_NAME}/trained_models/{EXP_NAME}"
RETRIEVED_DATA_NAME = "e001-train_ret25_map0.1378_recall0.4531.csv"

# experiment parameter
DEBUG = False
TRAINING = True
UPLOAD_DATA_TO_S3 = True
UPLOAD_DATA_TO_KAGGLE = True
WANDB = True

# model parameter
TRAINING_MAX_LENGTH = 128  # TODO: 十分な長さかどうかは調査が必要
INFERENCE_MAX_LENGTH = 128
SEED = 42
VALID_DATA_SIZE = 0.3
EPOCH = 4
LR = 2e-04
TRAIN_BS = 8
GRAD_ACC_NUM = 128 // TRAIN_BS  # 仮想的なバッチサイズはTRAIN_BS * GRAD_ACC_STEPとなる
EVAL_BS = 8
NUM_LABELS = 2  # regressionの場合は1, 2値分類の場合は2に設定する

In [2]:
!nvidia-smi

Sat Sep 14 05:16:45 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.161.07             Driver Version: 535.161.07   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-PCIE-16GB           On  | 00000000:E1:00.0 Off |                    0 |
| N/A   20C    P0              23W / 250W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
!python --version

Python 3.10.14


In [4]:
def resolve_path(base_path: str) -> str:
    import os

    cwd = os.getcwd()
    print(cwd)
    if cwd == f"/notebooks":
        print("Jupyter Kernel By VSCode!")
        return f"/notebooks/{COMPETITION_NAME}/{base_path}"
    elif cwd == f"/notebooks/{COMPETITION_NAME}":
        print("nohup!")
        return base_path
    elif cwd == f"/notebooks/{COMPETITION_NAME}/{COMPETITION_NAME}/exp":
        print("Jupyter Lab!")
        return f"../../{base_path}"
    elif cwd == f"/root/{COMPETITION_NAME}/exp/reranker":
        print("VastAi! Reranker")
        return f"../../{base_path}"
    elif cwd == f"/root/{COMPETITION_NAME}/exp/retriever":
        print("VastAi! Retriever")
        return f"../../{base_path}"
    else:
        raise Exception("Unknown environment")


DATA_PATH = resolve_path(DATA_PATH)
print(DATA_PATH)
MODEL_OUTPUT_PATH = resolve_path(MODEL_OUTPUT_PATH)
print(MODEL_OUTPUT_PATH)
ENV_PATH = resolve_path(ENV_PATH)
print(ENV_PATH)

/root/eedi-mining-misconceptions-in-mathematics/exp/reranker
VastAi! Reranker
../../retrieved_data
/root/eedi-mining-misconceptions-in-mathematics/exp/reranker
VastAi! Reranker
../../eedi-mining-misconceptions-in-mathematics/trained_models/e001-baseline
/root/eedi-mining-misconceptions-in-mathematics/exp/reranker
VastAi! Reranker
../../env_file


In [5]:
def validate_dataset_name(dataset_name: str) -> None:
    if len(dataset_name) < 6 or len(dataset_name) > 50:
        raise Exception(
            f"データセットの文字列は6~50文字にしてください。現在{len(DATASET_NAME)}文字"
        )
    if "_" in dataset_name:
        raise Exception("datasetの名称に_の使用は禁止です")


validate_dataset_name(DATASET_NAME)

# install

In [6]:
# %pip install -qq polars==1.7.1
# %pip install -qq transformers==4.44.2
# %pip install -qq sentencepiece==0.2.0
# %pip install -qq datasets==3.0.0
# %pip install -qq evaluate==0.4.3
# %pip install -qq seqeval==1.2.2
# %pip install -qq accelerate==0.34.2
# %pip install -qq python-dotenv==1.0.1
# %pip install -qq wandb==0.18.0
# %pip install -qq kaggle

# import

In [7]:
import os
import random
import ast
import json

import polars as pl
import numpy as np
import torch
import torch.nn as nn
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss
import wandb
from datasets import (
    Dataset,
    DatasetDict,
    Value,
    concatenate_datasets,
    load_dataset,
    ClassLabel,
)
from sklearn.metrics import cohen_kappa_score
from tokenizers import AddedToken
from tqdm.auto import tqdm
from scipy.special import softmax
from sklearn.metrics import log_loss
from transformers import (
    AutoConfig,
    AutoModel,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    DebertaV2PreTrainedModel,
    Trainer,
    TrainingArguments,
)
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers.models.deberta_v2.modeling_deberta_v2 import (
    ContextPooler,
    StableDropout,
    DebertaV2Model,
)

In [8]:
os.environ["TOKENIZERS_PARALLELISM"] = "true"
NUM_PROC = os.cpu_count()

In [9]:
import transformers
import datasets
import evaluate

assert transformers.__version__ == "4.44.2"
assert datasets.__version__ == "3.0.0"
assert evaluate.__version__ == "0.4.3"

In [10]:
# Seed the same seed to all
def seed_everything(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True


seed_everything(SEED)

In [11]:
from dotenv import load_dotenv

load_dotenv(f"{ENV_PATH}/.env")

True

# Wandb

In [12]:
if WANDB:
    wandb.login(key=os.environ["WANDB_API_KEY"])
    wandb.init(project=COMPETITION_NAME, name=EXP_NAME)
    REPORT_TO = "wandb"
else:
    REPORT_TO = "none"

REPORT_TO

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: sinchir0. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


'wandb'

# Data Import & Preprocess

In [13]:
train = pl.read_csv(f"{DATA_PATH}/{RETRIEVED_DATA_NAME}").rename({"target": "label"})

In [14]:
if DEBUG:
    train = pl.concat(
        [
            train.filter(pl.col("label") == 0).sample(fraction=1.0).head(50),
            train.filter(pl.col("label") == 1).sample(fraction=1.0).head(50),
        ]
    )

In [15]:
train_dataset = Dataset.from_polars(train)

In [16]:
train_dataset

Dataset({
    features: ['QuestionId', 'ConstructName', 'SubjectName', 'QuestionText', 'CorrectAnswer', 'AnswerType', 'AnswerText', 'AnswerAlphabet', 'MisconceptionId', 'MisconceptionAlphabet', 'AllText', 'PredictMisconceptionId', 'label', 'PredictMisconceptionName'],
    num_rows: 100
})

In [17]:
AutoTokenizer.from_pretrained(MODEL_NAME)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


DebertaV2TokenizerFast(name_or_path='microsoft/deberta-v3-xsmall', vocab_size=128000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	128000: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [18]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.add_tokens([AddedToken("\n", normalized=False)])
tokenizer.add_tokens([AddedToken(" " * 2, normalized=False)])

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=NUM_LABELS
)
model.resize_token_embeddings(len(tokenizer))

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, pad_to_multiple_of=16)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-xsmall and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Tokenize

In [19]:
train_dataset

Dataset({
    features: ['QuestionId', 'ConstructName', 'SubjectName', 'QuestionText', 'CorrectAnswer', 'AnswerType', 'AnswerText', 'AnswerAlphabet', 'MisconceptionId', 'MisconceptionAlphabet', 'AllText', 'PredictMisconceptionId', 'label', 'PredictMisconceptionName'],
    num_rows: 100
})

In [20]:
def tokenize(examples, max_token_length: int):
    separator = " [SEP] "

    joined_text = (
        examples["ConstructName"]
        + separator
        + examples["SubjectName"]
        + separator
        + examples["QuestionText"]
        + separator
        + examples["AnswerText"]
        + separator  # TODO: ここもSEPで良いかどうか
        + examples["PredictMisconceptionName"]
    )

    return tokenizer(
        joined_text,
        max_length=max_token_length,
        truncation=True,
        padding="max_length",
    )


train_dataset = train_dataset.map(
    tokenize,
    batched=False,
    fn_kwargs={"max_token_length": TRAINING_MAX_LENGTH},
    num_proc=NUM_PROC,
)

num_proc must be <= 100. Reducing num_proc to 100 for dataset of size 100.
Setting TOKENIZERS_PARALLELISM=false for forked processes.


Map (num_proc=100):   0%|          | 0/100 [00:00<?, ? examples/s]

In [21]:
print(tokenizer.decode(train_dataset["input_ids"][0]))

[CLS] Use the order of operations to carry out calculations involving powers[SEP] BIDMAS[SEP] \[
 3 \times 2+4-5
 \]
 Where do the brackets need to go to make the answer equal \( 13 \)?[SEP] Does not need brackets[SEP] Answers order of operations questions with brackets as if the brackets are not there[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]


In [22]:
print(tokenizer.decode(train_dataset["input_ids"][50]))

[CLS] Calculate the range from a list of data[SEP] Range and Interquartile Range from a List of Data[SEP] Tom and Katie are discussing the \( 5 \) plants with these heights:
 \( 24 \mathrm{~cm}, 17 \mathrm{~cm}, 42 \mathrm{~cm}, 26 \mathrm{~cm}, 13 \mathrm{~cm} \)
 Tom says if all the plants were cut in half, the range wouldn't change.
 Katie says if all the plants grew by \( 3 \mathrm{~cm} \)[SEP]


# Train Test Split

In [23]:
def to_train_valid(ds):
    return DatasetDict({"train": ds["train"], "valid": ds["test"]})


train_dataset = train_dataset.cast_column("label", ClassLabel(num_classes=NUM_LABELS))

train_valid_dataset = to_train_valid(
    (
        train_dataset.train_test_split(
            test_size=VALID_DATA_SIZE, seed=SEED, stratify_by_column="label"
        )
    )
)

Casting the dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

In [24]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds_prob = softmax(predictions, axis=-1)
    return {"log_loss": log_loss(labels, preds_prob)}

In [25]:
# スケジューラの設定
training_args = TrainingArguments(
    output_dir=MODEL_OUTPUT_PATH,
    learning_rate=LR,
    per_device_train_batch_size=TRAIN_BS,
    gradient_accumulation_steps=GRAD_ACC_NUM,
    eval_accumulation_steps=GRAD_ACC_NUM,
    per_device_eval_batch_size=EVAL_BS,
    num_train_epochs=EPOCH,
    weight_decay=0.01,
    eval_strategy="steps",
    eval_steps=0.1,
    save_strategy="steps",
    save_steps=0.1,
    save_total_limit=1,
    logging_steps=2,
    seed=SEED,
    metric_for_best_model="eval_loss",
    warmup_ratio=0.1,
    lr_scheduler_type="cosine_with_restarts",
    report_to=REPORT_TO,
    run_name=EXP_NAME,
    load_best_model_at_end=True,
    fp16=True,
    fp16_full_eval=True,
    gradient_checkpointing=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_valid_dataset["train"],
    eval_dataset=train_valid_dataset["valid"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [26]:
trainer.train()
# ログの保存に利用したストレージを削除
os.system(f"rm -rf {MODEL_OUTPUT_PATH}/checkpoint-*")
# モデルの保存
trainer.save_model(MODEL_OUTPUT_PATH)

/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Log Loss,Runtime,Samples Per Second,Steps Per Second
1,No log,0.692330,0.692330,0.096300,311.468000,41.529000
2,0.346400,0.677613,0.677614,0.101700,294.960000,39.328000
3,0.346400,0.668373,0.668373,0.087000,344.773000,45.970000
4,0.231800,0.636729,0.636729,0.101500,295.585000,39.411000


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning

# valid_datasetの作成・保存

In [27]:
# TRAININGをINFERRENCEでMAX_TOKENを変えるために、validを作り直す
valid_dataset = train_dataset.filter(
    lambda example: example["QuestionId"] in train_valid_dataset["valid"]["QuestionId"],
    num_proc=NUM_PROC,
)

valid_dataset = valid_dataset.map(
    tokenize,
    batched=False,
    fn_kwargs={"max_token_length": INFERENCE_MAX_LENGTH},
    num_proc=NUM_PROC,
)


def add_valid_pred(example, idx, valid_pred):
    example["valid_pred"] = valid_pred[idx]
    return example


valid_pred = softmax(trainer.predict(valid_dataset).predictions, axis=-1)

np.save(f"{MODEL_OUTPUT_PATH}/valid_prediction.npy", valid_pred)

valid_dataset = valid_dataset.map(
    add_valid_pred, with_indices=True, fn_kwargs={"valid_pred": valid_pred}
)

num_proc must be <= 100. Reducing num_proc to 100 for dataset of size 100.
Setting TOKENIZERS_PARALLELISM=false for forked processes.


Filter (num_proc=100):   0%|          | 0/100 [00:00<?, ? examples/s]

num_proc must be <= 76. Reducing num_proc to 76 for dataset of size 76.
Setting TOKENIZERS_PARALLELISM=false for forked processes.


Map (num_proc=76):   0%|          | 0/76 [00:00<?, ? examples/s]

Map:   0%|          | 0/76 [00:00<?, ? examples/s]

# CVの計算

In [41]:
valid_data_for_cv = (
    valid_dataset.to_polars()
    .with_columns(
        pl.col("valid_pred").map_elements(lambda x: x[1], return_dtype=pl.Float64)
    )
    .sort(by=["QuestionId", "valid_pred"], descending=[False, True])
    .group_by(["QuestionId", "AnswerAlphabet"])
    .agg(pl.col("PredictMisconceptionId").alias("Predict"))
).join(
    valid_dataset.to_polars()[["QuestionId", "AnswerAlphabet", "MisconceptionId"]],
    on=["QuestionId", "AnswerAlphabet"],
)
valid_data_for_cv

QuestionId,AnswerAlphabet,Predict,MisconceptionId
i64,str,list[i64],i64
0,"""D""","[2551, 256, … 1756]",1672
0,"""D""","[2551, 256, … 1756]",1672
0,"""D""","[2551, 256, … 1756]",1672
0,"""D""","[2551, 256, … 1756]",1672
0,"""D""","[2551, 256, … 1756]",1672
…,…,…,…
35,"""C""",[1965],1965
35,"""D""",[1973],1973
43,"""A""",[381],381


In [44]:
# https://www.kaggle.com/code/cdeotte/how-to-train-open-book-model-part-1#MAP@3-Metric
def map_at_25(predictions, labels):
    map_sum = 0
    for x, y in zip(predictions, labels):
        z = [1 / i if y == j else 0 for i, j in zip(range(1, 26), x)]
        map_sum += np.sum(z)
    return map_sum / len(predictions)


map_at_25_score = map_at_25(
    valid_data_for_cv["Predict"], valid_data_for_cv["MisconceptionId"]
)
print(f"MAP@25 Score: {map_at_25_score}")

MAP@25 Score: 0.34210526315789475


In [45]:
# output_textを保存
with open(f"{MODEL_OUTPUT_PATH}/cv_score.txt", "w") as f:
    f.write(str(map_at_25_score))

# AWSへのアップロード

In [48]:
# S3へのアップロード
if not DEBUG and UPLOAD_DATA_TO_S3:
    # uninstall
    !sudo rm /usr/bin/aws
    !sudo rm /usr/bin/aws_completer
    !sudo rm -rf /usr/local/aws-cli

    # install
    !curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
    !unzip -o -qq awscliv2.zip
    !sudo ./aws/install --update

    # upload
    output_name = MODEL_OUTPUT_PATH.split("/")[-1]
    os.system(
        f"aws s3 cp --recursive {MODEL_OUTPUT_PATH} s3://{COMPETITION_NAME}/trained_model/{output_name}"
    )

rm: cannot remove '/usr/bin/aws': No such file or directory
rm: cannot remove '/usr/bin/aws_completer': No such file or directory
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 58.0M  100 58.0M    0     0  71.3M      0 --:--:-- --:--:-- --:--:-- 71.3M
You can now run: /usr/local/bin/aws --version


upload failed: ../../eedi-mining-misconceptions-in-mathematics/trained_models/e001-baseline/config.json to s3://eedi-mining-misconceptions-in-mathematics/trained_model/e001-baseline/config.json An error occurred (NoSuchBucket) when calling the PutObject operation: The specified bucket does not exist
upload failed: ../../eedi-mining-misconceptions-in-mathematics/trained_models/e001-baseline/cv_score.txt to s3://eedi-mining-misconceptions-in-mathematics/trained_model/e001-baseline/cv_score.txt An error occurred (NoSuchBucket) when calling the PutObject operation: The specified bucket does not exist
upload failed: ../../eedi-mining-misconceptions-in-mathematics/trained_models/e001-baseline/added_tokens.json to s3://eedi-mining-misconceptions-in-mathematics/trained_model/e001-baseline/added_tokens.json An error occurred (NoSuchBucket) when calling the PutObject operation: The specified bucket does not exist
upload failed: ../../eedi-mining-misconceptions-in-mathematics/trained_models/e001-

upload failed: ../../eedi-mining-misconceptions-in-mathematics/trained_models/e001-baseline/valid_prediction.npy to s3://eedi-mining-misconceptions-in-mathematics/trained_model/e001-baseline/valid_prediction.npy An error occurred (NoSuchBucket) when calling the PutObject operation: The specified bucket does not exist


In [49]:
# ダウンロード（参考）
# !sudo rm /usr/bin/aws
# !sudo rm /usr/bin/aws_completer
# !sudo rm -rf /usr/local/aws-cli

# !curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
# !unzip -o -qq awscliv2.zip
# !sudo ./aws/install --update

# !aws s3 cp --recursive s3://automated-essay-scoring/trained_model/e005-regression /notebooks/automated_essay_scoring/trained_models/e005-regression

# Kaggle Datasetへのupload

In [50]:
if not DEBUG and UPLOAD_DATA_TO_KAGGLE:
    import os
    import json

    from kaggle.api.kaggle_api_extended import KaggleApi

    def dataset_create_new(dataset_name: str, upload_dir: str):
        # if "_" in dataset_name:
        #     raise ValueError("datasetの名称に_の使用は禁止です")
        dataset_metadata = {}
        dataset_metadata["id"] = f"sinchir0/{dataset_name}"
        dataset_metadata["licenses"] = [{"name": "CC0-1.0"}]
        dataset_metadata["title"] = dataset_name
        with open(os.path.join(upload_dir, "dataset-metadata.json"), "w") as f:
            json.dump(dataset_metadata, f, indent=4)
        api = KaggleApi()
        api.authenticate()
        api.dataset_create_new(folder=upload_dir, convert_to_csv=False, dir_mode="tar")

    print(f"Create Dataset name:{DATASET_NAME}, output_dir:{MODEL_OUTPUT_PATH}")
    dataset_create_new(dataset_name=DATASET_NAME, upload_dir=MODEL_OUTPUT_PATH)

Create Dataset name:e001-baseline-deberta-v3-xsmall, output_dir:../../eedi-mining-misconceptions-in-mathematics/trained_models/e001-baseline
Starting upload for file config.json


100%|██████████| 881/881 [00:00<00:00, 938B/s]  


Upload successful: config.json (881B)
Starting upload for file model.safetensors


100%|██████████| 270M/270M [01:20<00:00, 3.50MB/s] 


Upload successful: model.safetensors (270MB)
Starting upload for file tokenizer_config.json


...

Upload successful: training_args.bin (5KB)
Starting upload for file valid_prediction.npy


100%|██████████| 736/736 [00:00<00:00, 1.67kB/s]


Upload successful: valid_prediction.npy (736B)
Starting upload for file cv_score.txt


100%|██████████| 19.0/19.0 [00:00<00:00, 43.9B/s]


Upload successful: cv_score.txt (19B)


# ローカルからのデータの削除

In [ ]:
# if not DEBUG and (UPLOAD_DATA_TO_S3 or UPLOAD_DATA_TO_KAGGLE):
#     # ローカルからは削除
#     os.system(f"rm -rf {MODEL_OUTPUT_PATH}")

In [51]:
if WANDB:
    wandb.finish()

eval/log_loss,█▆▅▁
eval/loss,█▆▅▁
eval/runtime,▅█▁█
eval/samples_per_second,▃▁█▁
eval/steps_per_second,▃▁█▁
test/log_loss,▁
test/loss,▁
test/runtime,▁
test/samples_per_second,▁
test/steps_per_second,▁
train/epoch,▁▄▄▅███


In [52]:
print("finish Notebook!")

finish Notebook!
